# PREPARATIVOS

In [1]:
import pandas as pd
import numpy as np
import itertools as it
import re
import os
from funciones_1 import *
from funciones_2 import *
from importlib import reload

In [2]:
%load_ext line_profiler

In [3]:
#FBL3N = leer_excel("FBL3N.xlsx")

In [4]:
#FBL5N = leer_excel("FBL5N.xlsx")

In [5]:
#KNA1 = leer_excel("KNA1.xlsx")

In [6]:
#%store FBL3N FBL5N KNA1

In [7]:
%store -r FBL3N FBL5N KNA1

In [8]:
#PROPUESTA = leer_excel("PROPUESTA.xlsx", None)

# PARTE 1

In [9]:
textos = FBL3N["Texto"]
FBL3N["RUT"] = textos.apply(detectar_rut)
FBL3N["N° Factura"] = textos.apply(detectar_n_factura)

RUT_FBL3N = FBL3N[pd.notna(FBL3N["RUT"])]
FACT_FBL3N = FBL3N[pd.isna(FBL3N["RUT"]) & pd.notna(FBL3N["N° Factura"])]

# PARTE 2

### Trabajar con calces de factura

In [10]:
FBL5N["Referencia"] = FBL5N["Referencia"].apply(lambda s: str(s))
FBL5N["Cuenta"] = FBL5N["Cuenta"][pd.notna(FBL5N["Cuenta"])].apply(lambda n: str(int(n)))

KNA1["Nº ident.fis.1"] = KNA1["Nº ident.fis.1"].apply(lambda s: str(s).lstrip("0"))
KNA1["Cliente"] = KNA1["Cliente"][pd.notna(KNA1["Cliente"])].apply(lambda s: str(int(s)))

In [11]:
# Calzar números de factura
filtro_fact = FBL5N["Referencia"].isin(FACT_FBL3N["N° Factura"])
FACT_FBL5N = FBL5N[filtro_fact]
NO_FACT_FBL5N = FBL5N[~filtro_fact]

In [12]:
# Calzar RUTs
RUT_ID_FBL3N = pd.merge(RUT_FBL3N, KNA1[["Nº ident.fis.1", "Cliente"]], left_on="RUT", right_on="Nº ident.fis.1")

filtro_id5 = NO_FACT_FBL5N["Cuenta"].isin(RUT_ID_FBL3N["Cliente"])
ID_FBL5N = NO_FACT_FBL5N[filtro_id5]
NN_FBL5N = NO_FACT_FBL5N[~filtro_id5]

filtro_id3 = RUT_ID_FBL3N["Cliente"].isin(ID_FBL5N["Cuenta"])
ID_FBL3N = RUT_ID_FBL3N[filtro_id3]
SOLO_RUT_FBL3N = RUT_ID_FBL3N[filtro_id3]

In [13]:
try:
    os.mkdir("RESULTADOS/")
except:
    pass

In [14]:
MATCHES_FACT = pd.merge(FACT_FBL3N, FACT_FBL5N, left_on="N° Factura", right_on="Referencia")
escribir_excel(FACT_FBL5N, "RESULTADOS/Facturas.xlsx")

Escribiendo nuevo archivo Excel: RESULTADOS/Facturas.xlsx
Archivo Excel escrito.


# PARTE 3

### Trabajar con calces de ID

In [15]:
#MATCHES_ID_1_1 = match(ID_FBL3N, ID_FBL5N)
#MATCHES_ID_1_1

In [16]:
#escribir_excel(MATCHES_ID_1_1, "RESULTADOS/RUT y Monto 1-1.xlsx")

## Matches uno-a-uno

In [43]:
D = calzar(ID_FBL3N, ID_FBL5N)

531
523
522
520
520


In [44]:
DF = {}

In [45]:
DF["1-1"] = []

for ID in D["1-1"]:
    datos = D["1-1"][ID]
    df = {
        "ID": np.repeat(ID, datos["doc3"].size),
        "Documento FBL3N": datos["doc3"],
        "Monto FBL3N": datos["mon3"],
        "Documento FBL5N": datos["doc5"],
        "Monto FBL5N": datos["mon5"],
        "Demora": datos["dem5"],
    }
    df = pd.DataFrame.from_dict(df)
    DF["1-1"].append(df)
    
DF["1-1"] = pd.concat(DF["1-1"])
DF["1-1"].sort_values(by="ID")

,ID,Documento FBL3N,Monto FBL3N,Documento FBL5N,Monto FBL5N,Demora
0,1202524,100421777.0,54402,2.500385e+09,54402,-12.0
0,1298694,100150736.0,34886,1.400069e+09,34886,209.0
0,1317961,100170398.0,1398012,2.500339e+09,1398012,23.0
0,1324183,100364988.0,2736,1.400003e+09,2736,323.0
0,1718811,100395070.0,287540,2.500359e+09,287540,34.0
0,1742844,100191776.0,1000000,1.400035e+09,1000000,266.0
0,1745241,100205436.0,127949,2.500277e+09,127949,5.0
0,3141394,100351359.0,180785,2.500280e+09,180785,61.0
0,5003423,100096325.0,26656,2.500087e+09,26656,253.0


In [46]:
DF["1-T"] = []

for ID in D["1-T"]:
    datos = D["1-T"][ID]
    n = datos["doc5"].size
    df = {
        "ID": np.repeat(ID, n),
        "Documento FBL3N": np.repeat(datos["doc3"], n),
        "Monto FBL3N": np.repeat(datos["mon3"], n),
        "Documento FBL5N": datos["doc5"],
        "Monto FBL5N": datos["mon5"],
        "Demora": datos["dem5"],
    }
    df = pd.DataFrame.from_dict(df)
    DF["1-T"].append(df)
    
DF["1-T"] = pd.concat(DF["1-T"])
DF["1-T"].sort_values(by="ID")

,ID,Documento FBL3N,Monto FBL3N,Documento FBL5N,Monto FBL5N,Demora
0,3343520,100386142.0,23347,1.400168e+09,-95989,42.0
1,3343520,100386142.0,23347,2.500352e+09,119336,40.0


In [49]:
DF["1-V"] = []

for ID in D["1-V"]:
    datos = D["1-V"][ID]
    n = datos["doc5"].size
    df = {
        "ID": np.repeat(ID, n),
        "Documento FBL3N": np.repeat(datos["doc3"], n),
        "Monto FBL3N": np.repeat(datos["mon3"], n),
        "Documento FBL5N": datos["doc5"],
        "Monto FBL5N": datos["mon5"],
        "Demora": datos["dem5"],
    }
    df = pd.DataFrame.from_dict(df)
    DF["1-V"].append(df)
    
DF["1-V"] = pd.concat(DF["1-V"])
DF["1-V"].sort_values(by="ID")

,ID,Documento FBL3N,Monto FBL3N,Documento FBL5N,Monto FBL5N,Demora
0,1332355,100435197.0,688615,2.500332e+09,184420,27.0
16,1332355,100435197.0,688615,2.500351e+09,128853,10.0
15,1332355,100435197.0,688615,2.500347e+09,30807,13.0
14,1332355,100435197.0,688615,2.500347e+09,86832,13.0
13,1332355,100435197.0,688615,1.400185e+09,-875227,14.0
12,1332355,100435197.0,688615,2.500340e+09,45018,19.0
10,1332355,100435197.0,688615,2.500335e+09,45416,25.0
9,1332355,100435197.0,688615,2.500335e+09,53064,25.0
11,1332355,100435197.0,688615,2.500337e+09,78016,24.0
7,1332355,100435197.0,688615,2.500334e+09,653,26.0


In [20]:
pd.DataFrame.from_dict({clave: D5["1-M"][clave] for clave in ("ID", "Documento", "Monto", "Demora")})

NameError: name 'D5' is not defined

# PARTE 4

### Determinar qué datos encontré yo que Roberto no, y viceversa

1. RUT y Monto

In [ ]:
RUT_Y_MONTO = leer_excel("PROPUESTA/RUT y Monto.xlsx")

In [ ]:
outer_rym = pd.merge(MATCHES_ID_1_1, RUT_Y_MONTO, on="Nº documento", how="outer", indicator=True)

In [ ]:
try:
    os.mkdir("COMPARACIONES/")
except:
    pass

In [ ]:
escribir_excel(outer_rym[outer_rym["_merge"] == "left_only"], "COMPARACIONES/RUT y Monto - LEFT.xlsx")

In [ ]:
escribir_excel(outer_rym[outer_rym["_merge"] == "right_only"], "COMPARACIONES/RUT y Monto - RIGHT.xlsx")

2. Solo RUT

In [ ]:
SOLO_RUT = leer_excel("PROPUESTA/Solo RUT.xlsx")

In [ ]:
outer_sr = pd.merge(MATCHES_ID, SOLO_RUT, on="Nº documento", how="outer", indicator=True)

In [ ]:
escribir_excel(outer_sr[outer_sr["_merge"] == "left_only"], "SOLO_RUT_LEFT.xlsx")

In [ ]:
escribir_excel(outer_sr[outer_sr["_merge"] == "right_only"], "SOLO_RUT_RIGHT.xlsx")